In [7]:
import cv2
import opencv_jupyter_ui as jcv2
from feat import Detector
from IPython.display import Image


from feat.utils import FEAT_EMOTION_COLUMNS

In [8]:
detector = Detector(device="cuda")

INFO:root:Loading Face model: retinaface
c:\Users\Sophie\AppData\Local\Programs\Python\Python312\Lib\site-packages\feat\face_detectors\Retinaface\Retinaface_test.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issue

RuntimeError: unexpected EOF, expected 26690469 more bytes. The file might be corrupted.

In [ ]:
import os

# Construct the path to the image folder
ori_folder_path = os.path.join( '.', 'archive', 'test')




In [ ]:
    


# AU codes
predicted_aus = ["AU1","AU2","AU4","AU5","AU6","AU7","AU9","AU10","AU11","AU12","AU14","AU15",
                    "AU17","AU20","AU23","AU24","AU25","AU26","AU28", "AU43"]

import pandas as pd
df = pd.DataFrame(columns=['file', 'face']+predicted_aus)

total_images = 0
correct_images = 0
for folder_path in ori_folder_path:
    # load all the images in the folder
    images_filename = os.listdir(folder_path)
    # loop through all the images
    for image_filename in images_filename:
        frame = cv2.imread(ori_folder_path + "/" + image_filename)
        faces = detector.detect_faces(frame)
        landmarks = detector.detect_landmarks(frame, faces)
        emotions = detector.detect_emotions(frame, faces, landmarks)
        AUs = detector.detect_aus(frame, landmarks)
        faces = faces[0]
        landmarks = landmarks[0]
        emotions = emotions[0]
        AUs = AUs[0]

        strongest_emotion = emotions.argmax(axis=1)
        total_images += 1
        if FEAT_EMOTION_COLUMNS[strongest_emotion[0]] in folder_path:
            correct_images += 1

# save the results to a csv file
df.to_csv("results_pyfeat", index=False)

In [ ]:
# accoring to dataset/annotations.csv, separate the images into positive and negative

# load the annotations
annotations = pd.read_csv(os.path.join('..', 'dataset', 'annotations.csv'))
positives = df[df['file'].isin(annotations[annotations['valence'] == 'positive']['file'])]
negatives = df[df['file'].isin(annotations[annotations['valence'] == 'negative']['file'])]

# For each AU, calculate the absolute difference between the positive mean and the negative mean
diffs = {}
for au in predicted_aus:
    pos_mean = positives[au].mean()
    neg_mean = negatives[au].mean()
    diff = abs(pos_mean - neg_mean)
    diffs[au] = diff

# sort the AUs by the difference
sorted_diffs = sorted(diffs.items(), key=lambda x: x[1], reverse=True)
print(sorted_diffs)


In [ ]:
# au_visualization.png
# The x axis must correspond to which AU is being plotted, in the sorted order. Each data point must be correctly labeled with the name of the AU.
# The y axis must correspond to the absolute difference of means for that AU.
# By default, plt.plot() connects all the points as if they belonged to a continuous function. 
# Given the discrete nature of the x axis, we recommend using individual markers to display each data point
import matplotlib.pyplot as plt
import numpy as np

# get the AUs and the differences
aus, diffs = zip(*sorted_diffs)

# plot the AUs and the differences
plt.figure(figsize=(10, 5))
plt.plot(aus, diffs, marker='o', linestyle='None')
plt.xlabel('Action Units')
plt.ylabel('Absolute Difference of Means')
plt.title('Absolute Difference in AU Means (Pos. vs Neg.)')
au_visualization_path = os.path.join('..', 'processed', 'au_visualization.png') 
plt.savefig(au_visualization_path)


In [ ]:
print(FEAT_EMOTION_COLUMNS)